The process of creating a color detection project using OpenCV can be broken down into several steps. Here's a more detailed explanation of the process:

1. **Install OpenCV**: Ensure you have OpenCV installed in your Python environment. You can install it using `pip`:

   ```bash
   pip install opencv-python
   ```

2. **Import Necessary Libraries**: Import the required libraries, including OpenCV and NumPy, at the beginning of your Python script.

   ```python
   import cv2
   import numpy as np
   ```

3. **Initialize the Webcam**: Use OpenCV to initialize the webcam and start capturing video frames. You can specify the camera index (usually 0 for the default webcam) or a video file path if you want to analyze a pre-recorded video.

   ```python
   cap = cv2.VideoCapture(0)  # 0 represents the default webcam
   ```

4. **Define Color Range**: Specify the range of colors you want to detect in the HSV (Hue, Saturation, Value) color space. You need to set lower and upper bounds for the color you're interested in.

   ```python
   lower_color = np.array([30, 100, 100])  # Lower bound for the color (e.g., blue)
   upper_color = np.array([60, 255, 255])  # Upper bound for the color (e.g., blue)
   ```

   Adjust these values according to the specific color you want to detect.

5. **Capture and Process Frames**: Enter a loop that continuously captures frames from the webcam, processes each frame, and displays the original frame and the color detection result.

   - Read a frame from the webcam using `cap.read()`.
   - Convert the frame from BGR (Blue, Green, Red) color space to HSV using `cv2.cvtColor()`.
   - Create a mask using `cv2.inRange()` that isolates the pixels within the specified color range.
   - Apply the mask to the original frame using `cv2.bitwise_and()` to highlight the detected color.

   ```python
   while True:
       ret, frame = cap.read()
       hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
       color_mask = cv2.inRange(hsv_frame, lower_color, upper_color)
       result = cv2.bitwise_and(frame, frame, mask=color_mask)
       cv2.imshow('Original Frame', frame)
       cv2.imshow('Color Detection', result)
       if cv2.waitKey(1) & 0xFF == ord('q'):
           break
   ```

6. **Exit the Loop**: The loop continues until you press the 'q' key, at which point the script releases the webcam and closes all OpenCV windows.

   ```python
   cap.release()
   cv2.destroyAllWindows()
   ```

7. **Run the Script**: Save your script with a `.py` extension and run it using a Python interpreter. Make sure your webcam is connected and working.

8. **Interact with the Application**: When the script is running, your webcam feed will open, and you'll see the color detection in real-time. Move objects of the specified color in front of the camera to see the detection in action.

9. **Customization**: You can customize the script to detect different colors or adjust the color range for your specific needs. You can also add additional features, like drawing bounding boxes around detected objects or saving frames with detected colors to a file.

This process outlines the basic steps to create a color detection project using OpenCV and a webcam. You can further expand and enhance the project to suit your requirements and application.

In [2]:
import cv2
import numpy as np

# Create a video capture object for the default camera (0)
cap = cv2.VideoCapture(0)

# Callback function for trackbar adjustments (does nothing)
def nothing(x):
    pass

# Create a window for adjusting color parameters
cv2.namedWindow("Color Adjustments")

# Create trackbars to adjust lower and upper HSV values for color detection
cv2.createTrackbar("Lower_H", "Color Adjustments", 0, 255, nothing)
cv2.createTrackbar("Lower_S", "Color Adjustments", 0, 255, nothing)
cv2.createTrackbar("Lower_V", "Color Adjustments", 0, 255, nothing)
cv2.createTrackbar("Upper_H", "Color Adjustments", 255, 255, nothing)
cv2.createTrackbar("Upper_S", "Color Adjustments", 255, 255, nothing)
cv2.createTrackbar("Upper_V", "Color Adjustments", 255, 255, nothing)

while True:
    _,frame = cap.read()
    frame = cv2.resize(frame,(400,400))
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    #detecting hand
    l_h = cv2.getTrackbarPos("Lower_H", "Color Adjustments")
    l_s = cv2.getTrackbarPos("Lower_S", "Color Adjustments")
    l_v = cv2.getTrackbarPos("Lower_V", "Color Adjustments")

    u_h = cv2.getTrackbarPos("Upper_H", "Color Adjustments")
    u_s = cv2.getTrackbarPos("Upper_S", "Color Adjustments")
    u_v = cv2.getTrackbarPos("Upper_V", "Color Adjustments")
    
    lower_bound = np.array([l_h, l_s, l_v])
    upper_bound = np.array([u_h, u_s, u_v])
    
    #Creating Mask
    mask = cv2.inRange(hsv, lower_bound, upper_bound)
    #filter mask with image
    filtr = cv2.bitwise_and(frame, frame, mask=mask)
    
    
    mask1  = cv2.bitwise_not(mask)
    m_g = cv2.getTrackbarPos("Thresh", "Color Adjustments") #getting track bar value
    ret,thresh = cv2.threshold(mask1,m_g,255,cv2.THRESH_BINARY)
    dilata = cv2.dilate(thresh,(1,1),iterations = 6)
    
    
    
    #findcontour(img,contour_retrival_mode,method)
    cnts,hier = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    #Here cnts is a list of contours. ANd each contour is an array with x, y cordinate   
    #hier variable called hierarchy and it contain image information.
    #print("Number of contour==",cnts,"\ntotal contour==",len(cnts))
    #print("Hierarchy==\n",hier)
    
    #Draw the contours
    #frame= cv2.drawContours(frame,cnts,-1,(176,10,15),4)
    
    # loop over the contours
    
    for c in cnts:
        epsilon = 0.0001*cv2.arcLength(c,True)
        data= cv2.approxPolyDP(c,epsilon,True)
    
        hull = cv2.convexHull(data)
        cv2.drawContours(frame, [c], -1, (50, 50, 150), 2)
        cv2.drawContours(frame, [hull], -1, (0, 255, 0), 2)
        """
        hull = cv2.convexHull(data,returnPoints = False)
        defect = cv2.convexityDefects(data[0],hull)
        print("defect==",defect)
        """
    cv2.imshow("Thresh", thresh)
    cv2.imshow("mask==",mask)
    cv2.imshow("filter==",filtr)
    cv2.imshow("Result", frame)

    key = cv2.waitKey(25) &0xFF
    if key == 27:
        break
cap.release()
cv2.destroyAllWindows()
    

In [4]:
import cv2
import numpy as np 

#Read Camera
cap = cv2.VideoCapture(0)
def nothing(x):
    pass
#window name
cv2.namedWindow("Color Adjustments",cv2.WINDOW_NORMAL)
cv2.resizeWindow("Color Adjustments", (300, 300)) 
cv2.createTrackbar("Thresh", "Color Adjustments", 0, 255, nothing)

#COlor Detection Track

cv2.createTrackbar("Lower_H", "Color Adjustments", 0, 255, nothing)
cv2.createTrackbar("Lower_S", "Color Adjustments", 0, 255, nothing)
cv2.createTrackbar("Lower_V", "Color Adjustments", 0, 255, nothing)
cv2.createTrackbar("Upper_H", "Color Adjustments", 255, 255, nothing)
cv2.createTrackbar("Upper_S", "Color Adjustments", 255, 255, nothing)
cv2.createTrackbar("Upper_V", "Color Adjustments", 255, 255, nothing)


while True:
    _,frame = cap.read()
    frame = cv2.resize(frame,(400,400))
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    #detecting hand
    l_h = cv2.getTrackbarPos("Lower_H", "Color Adjustments")
    l_s = cv2.getTrackbarPos("Lower_S", "Color Adjustments")
    l_v = cv2.getTrackbarPos("Lower_V", "Color Adjustments")

    u_h = cv2.getTrackbarPos("Upper_H", "Color Adjustments")
    u_s = cv2.getTrackbarPos("Upper_S", "Color Adjustments")
    u_v = cv2.getTrackbarPos("Upper_V", "Color Adjustments")
    
    lower_bound = np.array([l_h, l_s, l_v])
    upper_bound = np.array([u_h, u_s, u_v])
    
    #Creating Mask
    mask = cv2.inRange(hsv, lower_bound, upper_bound)
    #filter mask with image
    filtr = cv2.bitwise_and(frame, frame, mask=mask)
    
    
    mask1  = cv2.bitwise_not(mask)
    m_g = cv2.getTrackbarPos("Thresh", "Color Adjustments") #getting track bar value
    ret,thresh = cv2.threshold(mask1,m_g,255,cv2.THRESH_BINARY)
    dilata = cv2.dilate(thresh,(1,1),iterations = 6)
    
    
    
    #findcontour(img,contour_retrival_mode,method)
    cnts,hier = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    #Here cnts is a list of contours. ANd each contour is an array with x, y cordinate   
    #hier variable called hierarchy and it contain image information.
    #print("Number of contour==",cnts,"\ntotal contour==",len(cnts))
    #print("Hierarchy==\n",hier)
    
    #Draw the contours
    #frame= cv2.drawContours(frame,cnts,-1,(176,10,15),4)
    
    # loop over the contours
    
    for c in cnts:
        epsilon = 0.0001*cv2.arcLength(c,True)
        data= cv2.approxPolyDP(c,epsilon,True)
    
        hull = cv2.convexHull(data)
        cv2.drawContours(frame, [c], -1, (50, 50, 150), 2)
        cv2.drawContours(frame, [hull], -1, (0, 255, 0), 2)
        # Calculate the average color of the detected object
        x, y, w, h = cv2.boundingRect(c)
        roi = frame[y:y+h, x:x+w]
        avg_color = np.mean(roi, axis=(0, 1)).astype(int)

        # Print the RGB values of the detected object
        print("Detected Object RGB Values:", avg_color)
    cv2.imshow("Thresh", thresh)
    cv2.imshow("mask==",mask)
    cv2.imshow("filter==",filtr)
    cv2.imshow("Result", frame)

    key = cv2.waitKey(25) &0xFF
    if key == 27:
        break
cap.release()
cv2.destroyAllWindows()
    
    

Detected Object RGB Values: [  0 255   0]
Detected Object RGB Values: [  0 255   0]
Detected Object RGB Values: [  0 255   0]
Detected Object RGB Values: [  0 255   0]
Detected Object RGB Values: [  0 255   0]
Detected Object RGB Values: [  0 255   0]
Detected Object RGB Values: [  0 255   0]
Detected Object RGB Values: [  0 255   0]
Detected Object RGB Values: [  0 255   0]
Detected Object RGB Values: [  0 255   0]
Detected Object RGB Values: [  0 255   0]
Detected Object RGB Values: [  0 255   0]
Detected Object RGB Values: [  0 255   0]
Detected Object RGB Values: [  0 255   0]
Detected Object RGB Values: [  0 255   0]
Detected Object RGB Values: [  0 255   0]
Detected Object RGB Values: [  0 255   0]
Detected Object RGB Values: [  0 255   0]
Detected Object RGB Values: [  0 255   0]
Detected Object RGB Values: [  0 255   0]
Detected Object RGB Values: [  0 255   0]
Detected Object RGB Values: [  0 255   0]
Detected Object RGB Values: [  0 255   0]
Detected Object RGB Values: [  0 2

In [3]:
import cv2
import numpy as np

# Create a video capture object for the default camera (0)
cap = cv2.VideoCapture(0)

# Callback function for trackbar adjustments (does nothing)
def nothing(x):
    pass

# Create a window for adjusting color parameters
cv2.namedWindow("Color Adjustments")

# Create trackbars to adjust lower and upper HSV values for color detection
cv2.createTrackbar("Lower_H", "Color Adjustments", 0, 255, nothing)
cv2.createTrackbar("Lower_S", "Color Adjustments", 0, 255, nothing)
cv2.createTrackbar("Lower_V", "Color Adjustments", 0, 255, nothing)
cv2.createTrackbar("Upper_H", "Color Adjustments", 255, 255, nothing)
cv2.createTrackbar("Upper_S", "Color Adjustments", 255, 255, nothing)
cv2.createTrackbar("Upper_V", "Color Adjustments", 255, 255, nothing)

while True:
    # Read a frame from the video capture
    _, frame = cap.read()
    
    # Resize the frame for better display (optional)
    frame = cv2.resize(frame, (400, 400))

    # Convert the BGR frame to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Get the current trackbar positions for lower and upper HSV values
    l_h = cv2.getTrackbarPos("Lower_H", "Color Adjustments")
    l_s = cv2.getTrackbarPos("Lower_S", "Color Adjustments")
    l_v = cv2.getTrackbarPos("Lower_V", "Color Adjustments")
    u_h = cv2.getTrackbarPos("Upper_H", "Color Adjustments")
    u_s = cv2.getTrackbarPos("Upper_S", "Color Adjustments")
    u_v = cv2.getTrackbarPos("Upper_V", "Color Adjustments")
    
    # Create lower and upper bounds for the specified color range
    lower_bound = np.array([l_h, l_s, l_v])
    upper_bound = np.array([u_h, u_s, u_v])
    
    # Create a mask based on the color range
    mask = cv2.inRange(hsv, lower_bound, upper_bound)
    
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Process each detected contour
    for contour in contours:
        # Calculate the average color of the detected object
        x, y, w, h = cv2.boundingRect(contour)
        roi = frame[y:y+h, x:x+w]
        avg_color = np.mean(roi, axis=(0, 1)).astype(int)

        # Print the RGB values of the detected object
        print("Detected Object RGB Values:", avg_color)

        # Draw a bounding box around the detected object
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    # Display the original frame and the color-extracted result
    cv2.imshow("Original Frame", frame)

    # Wait for a key press and break the loop if the 'Esc' key is pressed
    key = cv2.waitKey(25) & 0xFF
    if key == 27:
        break

# Release the video capture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


Detected Object RGB Values: [136 146 147]
Detected Object RGB Values: [136 147 148]
Detected Object RGB Values: [136 147 148]
Detected Object RGB Values: [136 147 148]
Detected Object RGB Values: [137 148 149]
Detected Object RGB Values: [137 148 149]
Detected Object RGB Values: [137 148 149]
Detected Object RGB Values: [137 148 150]
Detected Object RGB Values: [137 148 150]
Detected Object RGB Values: [137 148 150]
Detected Object RGB Values: [138 149 150]
Detected Object RGB Values: [138 149 150]
Detected Object RGB Values: [138 149 150]
Detected Object RGB Values: [138 149 150]
Detected Object RGB Values: [138 149 150]
Detected Object RGB Values: [138 149 150]
Detected Object RGB Values: [138 149 150]
Detected Object RGB Values: [138 149 150]
Detected Object RGB Values: [138 149 150]
Detected Object RGB Values: [138 148 149]
Detected Object RGB Values: [137 147 148]
Detected Object RGB Values: [136 146 147]
Detected Object RGB Values: [136 146 147]
Detected Object RGB Values: [136 1